In [16]:
import pandas as pd
import numpy as np
from gestor_datos_climaticos import GestorDatosClimaticos
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch
from torch import device
from modelo_lstm import LSTM
import warnings
warnings.filterwarnings("ignore")


In [17]:
class EvaluadorModelo:
    def __init__(self):
        self.resultados = pd.DataFrame(columns=['Modelo', 'MAE_Entrenamiento', 'RMSE_Entrenamiento', 'MSE_Entrenamiento', 'R_Entrenamiento', 
                                              'MAE_Validacion', 'RMSE_Validacion', 'MSE_Validacion', 'R_Validacion'])

    def evaluar_modelo(self, modelo, X_entrenamiento, y_entrenamiento, X_validacion, y_validacion):
        # Entrenar el modelo
        modelo.fit(X_entrenamiento, y_entrenamiento)

        # Hacer predicciones
        predicciones_entrenamiento = modelo.predict(X_entrenamiento)
        predicciones_validacion = modelo.predict(X_validacion)

        # Calcular métricas
        mae_entrenamiento = mean_absolute_error(y_entrenamiento, predicciones_entrenamiento)
        rmse_entrenamiento = np.sqrt(mean_squared_error(y_entrenamiento, predicciones_entrenamiento))
        mse_entrenamiento = mean_squared_error(y_entrenamiento, predicciones_entrenamiento)
        r_entrenamiento = r2_score(y_entrenamiento, predicciones_entrenamiento)

        mae_validacion = mean_absolute_error(y_validacion, predicciones_validacion)
        rmse_validacion = np.sqrt(mean_squared_error(y_validacion, predicciones_validacion))
        mse_validacion = mean_squared_error(y_validacion, predicciones_validacion)
        r_validacion = r2_score(y_validacion, predicciones_validacion)

        # Obtener el nombre del modelo base
        nombre_modelo = modelo.estimator.__class__.__name__ if hasattr(modelo, 'estimator') else modelo.__class__.__name__

        # Agregar resultados al DataFrame
        nueva_fila = pd.DataFrame({
            'Modelo': [nombre_modelo],
            'MAE_Entrenamiento': [mae_entrenamiento],
            'RMSE_Entrenamiento': [rmse_entrenamiento],
            'MSE_Entrenamiento': [mse_entrenamiento],
            'R_Entrenamiento': [r_entrenamiento],
            'MAE_Validacion': [mae_validacion],
            'RMSE_Validacion': [rmse_validacion],
            'MSE_Validacion': [mse_validacion],
            'R_Validacion': [r_validacion]
        })
        self.resultados = pd.concat([self.resultados, nueva_fila], ignore_index=True)

    def evaluar_lstm(self, modelo, X_entrenamiento, y_entrenamiento, X_validacion, y_validacion, epocas=100, intervalo=100):
        # Asegúrate de que los datos estén en el formato correcto
        tensor_X_entrenamiento = torch.tensor(X_entrenamiento, dtype=torch.float32).to(device)
        tensor_y_entrenamiento = torch.tensor(y_entrenamiento, dtype=torch.float32).to(device)
        tensor_X_validacion = torch.tensor(X_validacion, dtype=torch.float32).to(device)
        tensor_y_validacion = torch.tensor(y_validacion, dtype=torch.float32).to(device)

        # Entrenar el modelo
        historial = entrenar(modelo, 0.01, tensor_X_entrenamiento, tensor_y_entrenamiento, 
                           tensor_X_validacion, tensor_y_validacion, epocas=epocas, intervalo=intervalo)

        # Hacer predicciones
        modelo.eval()
        with torch.no_grad():
            predicciones_entrenamiento = modelo(tensor_X_entrenamiento)
            predicciones_validacion = modelo(tensor_X_validacion)

        # Calcular métricas
        mae_entrenamiento = torch.mean(torch.abs(predicciones_entrenamiento - tensor_y_entrenamiento)).item()
        rmse_entrenamiento = torch.sqrt(torch.mean((predicciones_entrenamiento - tensor_y_entrenamiento) ** 2)).item()
        mse_entrenamiento = torch.mean((predicciones_entrenamiento - tensor_y_entrenamiento) ** 2).item()
        r_entrenamiento = r2_score(tensor_y_entrenamiento.cpu(), predicciones_entrenamiento.cpu())

        mae_validacion = torch.mean(torch.abs(predicciones_validacion - tensor_y_validacion)).item()
        rmse_validacion = torch.sqrt(torch.mean((predicciones_validacion - tensor_y_validacion) ** 2)).item()
        mse_validacion = torch.mean((predicciones_validacion - tensor_y_validacion) ** 2).item()
        r_validacion = r2_score(tensor_y_validacion.cpu(), predicciones_validacion.cpu())

        # Agregar resultados al DataFrame
        nueva_fila = pd.DataFrame({
            'Modelo': ['LSTM'],
            'MAE_Entrenamiento': [mae_entrenamiento],
            'RMSE_Entrenamiento': [rmse_entrenamiento],
            'MSE_Entrenamiento': [mse_entrenamiento],
            'R_Entrenamiento': [r_entrenamiento],
            'MAE_Validacion': [mae_validacion],
            'RMSE_Validacion': [rmse_validacion],
            'MSE_Validacion': [mse_validacion],
            'R_Validacion': [r_validacion]
        })
        self.resultados = pd.concat([self.resultados, nueva_fila], ignore_index=True)

    def mostrar_resultados(self):
        # Redondear las columnas numéricas a 3 decimales
        print(self.resultados.round(3))

In [18]:
# Inicializar el gestor de base de datos
ciudad = 'Piura'
gestor_db = GestorDatosClimaticos()
gestor_db.actualizar_datos_climaticos(ciudad)
id_ciudad = gestor_db.obtener_id_ciudad_por_nombre(ciudad)

# Obtener y procesar datos
df_clima = gestor_db.obtener_dataframe('clima', id_ciudad)
df_clima = df_clima.dropna(axis=1, how='all')
df_clima['fecha'] = pd.to_datetime(df_clima['time'])
df_clima['mes'] = df_clima['fecha'].dt.month

# Crear variables dummy para meses
meses_codificados = pd.get_dummies(df_clima['mes'], prefix='mes').astype(int)
df_clima = pd.concat([df_clima, meses_codificados], axis=1)
df_clima.drop('mes', axis=1, inplace=True)

La última fecha registrada en la tabla 'clima' para Piura es: 2024-12-26 00:00:00

La base de datos ya está actualizada para la ciudad Piura. No es necesario actualizar los datos.



In [19]:
# Definir características de entrada y variables objetivo
caracteristicas = [col for col in df_clima.columns if col not in ['id_ciudad', 'time','snow', 'wpgt', 'tsun', 'fecha']]
variables_objetivo = ['tmax', 'tmin', 'tavg']

# División temporal de datos
df_entrenamiento = df_clima[df_clima['time'] <= '2023-12-31']
df_validacion = df_clima[(df_clima['time'] > '2023-12-31') & (df_clima['time'] <= '2024-06-30')]
print(f'La data de entrenamiento tiene {len(df_entrenamiento)} filas')
print(f'La data de validación tiene {len(df_validacion)} filas')


La data de entrenamiento tiene 1991 filas
La data de validación tiene 182 filas


In [20]:
# Preparación de datos de entrada y salida
datos_entrada_entrenamiento = df_entrenamiento[caracteristicas].values
datos_entrada_validacion = df_validacion[caracteristicas].values
objetivos_entrenamiento = df_entrenamiento[variables_objetivo].values
objetivos_validacion = df_validacion[variables_objetivo].values

In [21]:
# Configuración de parámetros de ventana temporal
VENTANA_TIEMPO = 20  # T pasos de tiempo
DIMENSION_ENTRADA = datos_entrada_entrenamiento.shape[1]
N_ENTRENAMIENTO = len(datos_entrada_entrenamiento) - VENTANA_TIEMPO
N_VALIDACION = len(datos_entrada_validacion) - VENTANA_TIEMPO
DIMENSION_SALIDA = len(variables_objetivo)

In [22]:
# Normalización de datos
normalizador = StandardScaler()
# Excluir columnas de fecha antes de la normalización
normalizador.fit(datos_entrada_entrenamiento[:len(datos_entrada_entrenamiento) + VENTANA_TIEMPO - 1])
datos_entrada_entrenamiento = normalizador.transform(datos_entrada_entrenamiento)
datos_entrada_validacion = normalizador.transform(datos_entrada_validacion)

# Preparación de datos de entrenamiento
X_entrenamiento = np.zeros((N_ENTRENAMIENTO, VENTANA_TIEMPO, DIMENSION_ENTRADA))
y_entrenamiento = np.zeros((N_ENTRENAMIENTO, DIMENSION_SALIDA))

for t in range(N_ENTRENAMIENTO):
    X_entrenamiento[t, :, :] = datos_entrada_entrenamiento[t:t+VENTANA_TIEMPO]
    y_entrenamiento[t] = objetivos_entrenamiento[t+VENTANA_TIEMPO]

# Preparación de datos de validación
X_validacion = np.zeros((N_VALIDACION, VENTANA_TIEMPO, DIMENSION_ENTRADA))
y_validacion = np.zeros((N_VALIDACION, DIMENSION_SALIDA))

for i in range(N_VALIDACION):
    t = i
    X_validacion[i, :, :] = datos_entrada_validacion[t:t+VENTANA_TIEMPO]
    y_validacion[i] = objetivos_validacion[t+VENTANA_TIEMPO]

In [23]:
modelos = {
    'MLP': MLPRegressor(
        activation='relu',
        alpha=0.01,
        hidden_layer_sizes=(64,),
        learning_rate='constant',
        solver='sgd'
    ),
    'Linear Regression': MultiOutputRegressor(LinearRegression()),
    'Elastic Net': MultiOutputRegressor(ElasticNet(
        alpha=0.1,
        l1_ratio=0.5
    )),
    'Decision Tree': MultiOutputRegressor(DecisionTreeRegressor(
        max_depth=5,
        min_samples_leaf=3,
        min_samples_split=3
    )),
    'Random Forest': MultiOutputRegressor(RandomForestRegressor(
        max_depth=None,
        min_samples_leaf=3,
        min_samples_split=2,
        n_estimators=100
    )),
    'Gradient Boosting': MultiOutputRegressor(GradientBoostingRegressor(
        learning_rate=0.1,
        max_depth=3,
        n_estimators=50
    )),
    'XGBoost': MultiOutputRegressor(XGBRegressor(
        learning_rate=0.1,
        max_depth=3,
        n_estimators=50
    )),
    'SVR': MultiOutputRegressor(SVR(
        C=0.1,
        epsilon=0.2,
        kernel='linear'
    )),
}

In [24]:
# Configurar dispositivo
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dimensiones del modelo
dimension_entrada = len(caracteristicas)      # Número de características de entrada
dimension_oculta = 256                        # Dimensión de las capas ocultas
numero_capas = 1                              # Número de capas LSTM
dimension_salida = len(variables_objetivo)    # Número de variables a predecir
prob_dropout = 0.2                            # Probabilidad de desactivación de neuronas

# Crear y configurar el modelo LSTM
modelo = LSTM(dimension_entrada, dimension_oculta, numero_capas, dimension_salida, prob_dropout)

# Mover modelo al dispositivo de procesamiento
modelo.to(device)

LSTM(
  (lstm): LSTM(19, 256, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=3, bias=True)
)

In [25]:
# Reshape de datos para modelos tradicionales
X_entrenamiento_2D = X_entrenamiento.reshape(X_entrenamiento.shape[0], -1)
X_validacion_2D = X_validacion.reshape(X_validacion.shape[0], -1)

# Crear instancia del evaluador de modelos
evaluador = EvaluadorModelo()

# Evaluar modelo LSTM
evaluador.evaluar_lstm(modelo, X_entrenamiento, y_entrenamiento, 
                      X_validacion, y_validacion, epocas=300, intervalo=300)

# Evaluar modelos tradicionales
for nombre_modelo, modelo in modelos.items():
    evaluador.evaluar_modelo(modelo, X_entrenamiento_2D, y_entrenamiento, 
                           X_validacion_2D, y_validacion)

# Mostrar resultados comparativos
evaluador.mostrar_resultados()

Entrenando Modelo LSTM TOTAL: 100%|█████████████████████████████████████████████████| 300/300 epocas


Época 300/300:
Train Loss: 0.909, Val Loss: 2.187
Train MAE: 0.686, Val MAE: 1.132


                      Modelo  MAE_Entrenamiento  RMSE_Entrenamiento  \
0                       LSTM              0.685               0.953   
1               MLPRegressor              0.443               0.610   
2           LinearRegression              0.610               0.834   
3                 ElasticNet              0.667               0.931   
4      DecisionTreeRegressor              0.658               0.891   
5      RandomForestRegressor              0.294               0.468   
6  GradientBoostingRegressor              0.573               0.755   
7               XGBRegressor              0.576               0.767   
8                        SVR              0.589               0.871   

   MSE_Entrenamiento  R_Entrenamiento  MAE_Validacion  RMSE_Validacion  \
0              0.909            0.866           1.132            1.479   
1              0.372            0.946           1.452            1.855   
2              0.696            0.897           1.110            1.